In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt


/home/nwertheim/miniconda3/bin/python


In [2]:
data_file = os.path.join(univariate_data_path, 'merged_univariate.npy')
data = np.load(data_file, allow_pickle=True)
print(data.shape)

# Check for NaN or Inf values in signals
for sample in data:
    if np.isnan(sample['signal']).any() or np.isinf(sample['signal']).any():
        print(f"NaN or Inf detected in {sample['record_name']}")


data[0]['signal'].shape
print(data[0])

(666,)
{'record_name': 'ice001_l_1of1', 'signal': array([[-1.7358303 ],
       [-0.30347557],
       [-0.40749874],
       ...,
       [-3.09738299],
       [-2.90981482],
       [-3.22768386]]), 'metadata': {'fs': 20, 'sig_len': 100000, 'n_sig': 16, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'comments': ['Info:', 'ID:ice001', 'Record type:labour', 'Record number:1/1', 'Age(years):31', 'BMI before pregnancy:23.3', 'BMI at recording:27.6', 'Gravidity:3', 'Parity:2', 'Previous caesarean:No', 'Placental position:Fundus', 'Gestational age at recording(w/d):39/3', 'Gestational age at delivery:39/3', 'Mode of delivery:Vaginal', 'Synthetic oxytocin use in labour:No', 'Epidural during labour:No', 'Comments for recording:', 'Electrodes placed 5-10 mins prior to beginning of recording.', 'Baby born 20 minutes after the end of the recording.']}}


In [3]:
# Masking function using patch-based masking
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """ Apply patch-based masking to 1D signals """
    x_masked = np.copy(x)
    num_patches = x.shape[1] // patch_size  # Number of patches
    mask = np.random.rand(num_patches) < mask_ratio  # Randomly mask patches
    for i in range(num_patches):
        if mask[i]:
            x_masked[:, i * patch_size:(i + 1) * patch_size, :] = 0  # Zero out patches
    return x_masked

# Create windows from the data
def create_windows(sequence, window_size, step_size):
    windows = [sequence[i:i+window_size] for i in range(0, len(sequence) - window_size + 1, step_size)]
    return np.array(windows)

# Set windowing parameters
window_size = 500  
step_size = 250    

# Assuming `data` is a list of dictionaries with signal data
# Apply windowing to all records
all_windows = [create_windows(record['signal'], window_size, step_size) for record in data]
all_windows = np.concatenate(all_windows, axis=0)

# Reshape for Conv1D (batch_size, time_steps, channels)
all_windows = np.expand_dims(all_windows, axis=-1)  
print(f"Processed window shape: {all_windows.shape}")

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 275 has 1 dimension(s)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

class FullyConvolutionalMaskedAutoencoder(nn.Module):
    def __init__(self, input_channels=1, num_layers=3, filters=[64, 32, 16]):
        super().__init__()
        self.encoder = self.build_encoder(input_channels, num_layers, filters)
        self.decoder = self.build_decoder(input_channels, num_layers, filters)
    
    def build_encoder(self, input_channels, num_layers, filters):
        layers = []
        in_channels = input_channels
        for i in range(num_layers):
            layers.append(nn.Conv1d(in_channels, filters[i], kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            if i < num_layers - 1:
                layers.append(nn.MaxPool1d(kernel_size=2, stride=2))
            in_channels = filters[i]
        return nn.Sequential(*layers)
    
    def build_decoder(self, input_channels, num_layers, filters):
        layers = []
        in_channels = filters[-1]
        for i in range(num_layers):
            layers.append(nn.ConvTranspose1d(in_channels, filters[-(i + 1)], kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            if i < num_layers - 1:
                layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
            in_channels = filters[-(i + 1)]
        layers.append(nn.ConvTranspose1d(in_channels, input_channels, kernel_size=3, padding=1))
        layers.append(nn.Sigmoid())
        return nn.Sequential(*layers)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def extract_encoder(self):
        return self.encoder  # Returns only the encoder part


# Define training loop
def train_fcmae(model, train_data, val_data, epochs=50, batch_size=32, learning_rate=1e-4, device='cuda'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    
    for epoch in range(epochs):
        print('Epoch: ', epoch)
        model.train()
        train_loss = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")
    
    return model

# # Prepare data
# X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)
# X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
# X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])

# # Convert to PyTorch tensors
# tensor_X_train = torch.tensor(X_train_masked, dtype=torch.float32).permute(0, 2, 1)  # [batch, channels, seq_len]
# tensor_X_val = torch.tensor(X_val_masked, dtype=torch.float32).permute(0, 2, 1)
# tensor_Y_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)
# tensor_Y_val = torch.tensor(X_val, dtype=torch.float32).permute(0, 2, 1)


# # Create datasets & loaders
# train_dataset = TensorDataset(tensor_X_train, tensor_Y_train)
# val_dataset = TensorDataset(tensor_X_val, tensor_Y_val)

# # Train the model
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# autoencoder = FullyConvolutionalMaskedAutoencoder(input_channels=1, num_layers=3, filters=[64, 32, 16])
# trained_model = train_fcmae(autoencoder, train_dataset, val_dataset, epochs=1, batch_size=32, device=device)

# # Extract & save encoder
# encoder = autoencoder.extract_encoder()
# save_location = os.path.join(models_path, 'encoder_model.pth')
# torch.save(encoder.state_dict(), save_location)
# print("Saved encoder model!")


In [5]:
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import torch.nn.functional as F

# Define the search space
search_space = [
    Real(1e-5, 1e-2, "log-uniform", name='learning_rate'),
    Categorical([16, 32, 64], name='batch_size'),
    Integer(2, 5, name='num_layers'),
    Categorical([16, 32, 64], name='base_filters')  
]

# Create a function to build and train the model with hyperparameters
def build_model(learning_rate, num_layers, base_filters):
    input_shape = (500, 1)  # Shape of each time series window
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern
    
    # Ensure the number of input channels for the first layer is correct
    layers = []
    
    # Encoder layers
    in_channels = 1  # Input has 1 channel
    for i in range(num_layers):
        layers.append(nn.Conv1d(in_channels, filters[i], kernel_size=3, stride=1, padding=1))
        layers.append(nn.ReLU())
        in_channels = filters[i]  # Update input channels for next layer
    
    # Decoder layers (reverse the filters)
    for i in range(num_layers - 2, -1, -1):  # Skip the last filter in the encoder part
        layers.append(nn.ConvTranspose1d(in_channels, filters[i], kernel_size=3, stride=1, padding=1))
        layers.append(nn.ReLU())
        in_channels = filters[i]
    
    # Final layer (reconstruct to the original input size)
    layers.append(nn.Conv1d(in_channels, 1, kernel_size=3, stride=1, padding=1))
    
    # Combine all layers into a single sequential model
    model = nn.Sequential(*layers)
    return model


def objective(params):
    learning_rate, batch_size, num_layers, base_filters = params  # ✅ base_filters instead of filters_per_layer
    
    # Compute filter sizes
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern
    
    # Print the hyperparameters and filters for each layer
    print(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}")
    print(f"Filters per layer: {filters}")  # Print filter sizes for each layer
    
    # Build the model
    model = build_model(learning_rate, num_layers, base_filters)
    
    # Split the data into training and validation sets
    X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)
    
    # Apply masking to the data
    X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
    X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])
    
    # Convert to PyTorch tensors
    tensor_X_train = torch.tensor(X_train_masked, dtype=torch.float32).permute(0, 2, 1)  # [batch, channels, seq_len]
    tensor_X_val = torch.tensor(X_val_masked, dtype=torch.float32).permute(0, 2, 1)
    tensor_Y_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)
    tensor_Y_val = torch.tensor(X_val, dtype=torch.float32).permute(0, 2, 1)

    # Create datasets & loaders
    train_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_Y_train)
    val_dataset = torch.utils.data.TensorDataset(tensor_X_val, tensor_Y_val)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Set up optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(5):  # 5 epochs for tuning
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)
            val_loss += loss.item()
    
    # Print the validation loss to the console
    print(f"Validation Loss: {val_loss / len(val_loader)}")
    
    # Append the results to a file
    with open("FCMAE_tuning.txt", "a") as file:
        file.write(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}\n")
        file.write(f"Filters per layer: {filters}\n")  # Log filters for each layer
        file.write(f"Validation Loss: {val_loss / len(val_loader)}\n\n")
    
    return val_loss / len(val_loader)

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=15, random_state=42)

# Print the best hyperparameters and validation loss to the console
best_params = result.x
print(f"Best Hyperparameters: {best_params}")

with open("FCMAE_tuning.txt", "a") as file:
    file.write(f"Best Hyperparamters: {best_params}\n")


Learning Rate: 0.002452612631133679, Batch Size: 16, Num Layers: 4, Base Filters: 32
Filters per layer: [32, 64, 128, 256, 128, 64, 32]
